In [4]:
import sys
import time
import torch
import torch.nn.functional as F
from utils.utils import load_data, mat2tensor, regression_loss
from model.gcn import GCN
from model.gat import GAT
import numpy as np
import dgl

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load data
features_list, adjM, labels, train_val_test_idx, dl = load_data('iYO844')
features_list = [mat2tensor(features).to(device)
                    for features in features_list]
m_dim = features_list[1].shape[1]

# Set train, val, test index
labels = torch.FloatTensor(labels).to(device)
train_idx = train_val_test_idx['train_idx']
train_idx = np.sort(train_idx)
val_idx = train_val_test_idx['val_idx']
val_idx = np.sort(val_idx)
test_idx = train_val_test_idx['test_idx']
test_idx = np.sort(test_idx)

# Build graph
g = dgl.from_scipy(adjM+(adjM.T))
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)
g = g.to(device)

# Set model
num_labels = dl.labels_train['num_labels']

In [6]:
net = GCN(g, m_dim, 64, num_labels, 2, F.elu, 0.1)
net

GCN(
  (layers): ModuleList(
    (0): GraphConv(in=64, out=64, normalization=both, activation=<function elu at 0x7fa205772940>)
    (1): GraphConv(in=64, out=64, normalization=both, activation=<function elu at 0x7fa205772940>)
    (2): GraphConv(in=64, out=2, normalization=both, activation=None)
  )
  (e_conv): Sequential(
    (0): Conv2dBlock(
      (conv): Conv2d(1, 1, kernel_size=(4, 4), stride=(4, 4), bias=False)
      (batchnorm): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.1)
    )
    (1): Conv2dBlock(
      (conv): Conv2d(1, 1, kernel_size=(4, 4), stride=(4, 4), bias=False)
      (batchnorm): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.1)
    )
    (2): Conv2dBlock(
      (conv): Conv2d(1, 1, kernel_size=(4, 4), stride=(4, 4), bias=False)
      (batchnorm): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_run

In [7]:
result = net(features_list)

In [8]:
result.shape

torch.Size([1000, 2])

In [9]:
result

tensor([[ 0.5051, -0.6336],
        [ 0.9062, -1.1658],
        [ 0.9253, -1.0670],
        ...,
        [-0.1441, -0.8804],
        [-0.4972,  1.1266],
        [-0.1885, -1.4643]], grad_fn=<AddBackward0>)